<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_si_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition

In [2]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf

In [3]:
#!git clone https://github.com/bagustris/ravdess_song.git

In [4]:
#cd ravdess_song

In [5]:
# load extracted features
x_train = np.load('data/x_train.npy', allow_pickle=True)
x_test = np.load('data/x_test.npy', allow_pickle=True)
y_train = np.load('data/y_train.npy', allow_pickle=True)
y_test = np.load('data/y_test.npy', allow_pickle=True)


In [6]:
# reshape X for CNN
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


In [8]:
def model_cnn():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.Conv1D(256, 128, 1, padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [9]:
model = model_cnn()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 386, 1)            4         
_________________________________________________________________
conv1d (Conv1D)              (None, 386, 256)          33024     
_________________________________________________________________
activation (Activation)      (None, 386, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 193, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 49408)             0         
_________________________________________________________________
dropout (Dropout)            (None, 49408)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 2

In [10]:
# train CNN model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
53/53 [==============================] - 12s 187ms/step - loss: 1.7797 - accuracy: 0.2727
Epoch 2/100
53/53 [==============================] - 10s 196ms/step - loss: 1.5189 - accuracy: 0.3876
Epoch 3/100
53/53 [==============================] - 11s 211ms/step - loss: 1.4105 - accuracy: 0.4270
Epoch 4/100
53/53 [==============================] - 10s 196ms/step - loss: 1.2227 - accuracy: 0.5431
Epoch 5/100
53/53 [==============================] - 10s 185ms/step - loss: 1.1984 - accuracy: 0.5622
Epoch 6/100
53/53 [==============================] - 10s 192ms/step - loss: 1.0871 - accuracy: 0.5658
Epoch 7/100
53/53 [==============================] - 11s 202ms/step - loss: 0.9520 - accuracy: 0.6388
Epoch 8/100
53/53 [==============================] - 10s 195ms/step - loss: 0.9450 - accuracy: 0.6543
Epoch 9/100
53/53 [==============================] - 11s 199ms/step - loss: 0.8502 - accuracy: 0.6950
Epoch 10/100
53/53 [==============================] - 11s 207ms/step - loss: 0.716

In [11]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

11/11 [==============================] - 1s 52ms/step - loss: 0.8181 - accuracy: 0.7159
Loss:  0.818128228187561 --> Accuracy:  0.7159090638160706
